# <span style="color:orange">Nettoyer des jeux de données pour obtenir une liste de DOI des publications de l'Université de univ_paris8 : Web of Science, Pubmed, HAL, données des APC, Lens.org</span>

Ce premier notebook sert à nettoyer les différents fichiers obtenus après téléchargement sur le WoS, Pubmed, HAL, les données d'APC et Lens.org. Pour savoir quelques requêtes ont été utilisées pour l'Université de univ_paris8, consulter le fichier intitulé "requetes_bdd" dans le dossier. Quelques consignes sont à respecter pour que tout fonctionne :

- Pour le WoS, il suffit de procéder à un téléchargement simple "Fast 5000". Le fichier obtenu, en texte, est illisible et c'est normal, il n'y a rien à changer. Nommer le fichier "wos_univ_paris8_2016", puis "wos_univ_paris8_2017"... Ce fichier n'apparaît pas dans le dossier téléchargé depuis Gitlab car les données du Web of Science étant propriétaires, il n'était pas possible de les diffuser librement.
- Pour Scopus, télécharger uniquement le DOI : on obtient un fichier CSV brut avec une colonne DOI,Link.
- Pour Pubmed, le téléchargement donne un fichier CSV très peu classé, c'est normal, il n'y a rien à changer. Nommer le fichier "pubmed_univ_paris8_2016", puis "pubmed_univ_paris8_2017"...
- Pour les autres sources de données, on obtient directement une liste de DOI, mais **il faut s'assurer que la colonne s'appelle bien "doi" en minuscules et qu'il n'y a pas de ligne vide**

Il faut télécharger année par année, et toujours nommer les fichiers de la même manière.  <span style="color:red"> **Il est vital de garder l'organisation ici présente (Data > raw > dossier par année) pour que le code fonctionne.**</span>

<span style="color:red">Si l'on ne dispose pas de certaines données (par exemple, l'établissement n'a pas de données sur les APC ou n'utilise pas le Web of Science), il ne faut pas exécuter les parties de code liées à ces outils. Si l'on ne dispose pas d'extractions du Web of Science, on n'exécute pas toute la partie "Nettoyer les données issues du Web of Science".

**Il faut remplacer "univ_paris8" par le nom de l'établissement directement dans le code ci-dessous. Vous pouvez faire ctrl+f pour modifier toutes les occurrences d'un coup.**</span> Eviter de mettre des traits d'union qui génère des erreurs.

Commencer par exécuter les lignes ci-dessous : cliquer sur la ligne puis ensuite sur le bouton "play" de la barre d'outils.

In [1]:
column_name = "doi"

In [2]:
import pandas

In [3]:
import csv
import datetime
import re

## <span style="color:orange">Nettoyer les données issues du Web of Science</span>

### Comprendre comment est structuré le fichier

On ouvre le premier fichier puis on utilise la méthode de liste **split** pour voir une ligne (la 4ème, par exemple).

In [4]:
with open("Data/raw/2016/wos_univ_paris8_2016.txt") as f:
    wos_string = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'Data/raw/2016/wos_univ-paris8_2016.txt'

La ligne ci-dessous permet de voir à quoi ressemble la 5ème ligne du fichier (en Python, la numérotation commence à zéro).

In [5]:
wos_string.split("\n") [4]

'J\tMezin, Andre; Hemel-Manquin, Audrey\t\t\t\t\t\t\t\tDecohesion and rupture mechanisms of a multilayered microbattery studied by 4-point bending\t\t\t\t\t\t\t\tJOURNAL OF POWER SOURCES\t\t\t\t336\t\t\t\t150\t160\t\t10.1016/j.jpowsour.2016.10.035\t\t\tDEC 30 2016\t2016\tThis paper presents tests to characterize the mechanical resistance of the multilayers set which forms an all-solid-state lithium-ion micro battery. The system under investigation consists in five layers that were deposited successively onto a crystalline silicon substrate (100): Ti, TiOS, LiPON, aSi, Ti. The layer thicknesses are between 65 nm (amorphous silicon layer aSi) and 1400 nm (LiPON, TiOS). First, simple water immersion experiments allow internal stress to be put into evidence in the two external layers (aSi and Ti), basically a strong compressive stress in the aSi layer (at least 0.8 GPa). Afterwards, well controlled 4-point bending tests lead to delamination of the weakest parts of the multilayer, which are

### Définir une fonction

Définition de la fonction **read_wos** : l'année est en argument. On tape la 1ère année concernée (2016) pour que cela utilise les données de 2016 en premier. Dans cette fonction, les fichiers de toutes les années seront interrogés. 

On crée une liste vide (wos_per_year) dans laquelle on stockera après les données de toutes les années grâce à la boucle **for**.  <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2024. Il faudra mettre 2026 à la place de 2025 quand les données de 2025 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>

La liste vide wos_per_year est remplie à chaque tour de boucle grâce à la méthode **append**. Pour avoir la liste de toutes les données sans tri par année, on concatène les fichiers grâce à **pandas.concat**.

In [14]:
def read_wos(year):
    wos_per_year = []
    for year in range (2016, datetime.date.today().year):
       wos_df = pandas.read_csv("Data/raw/{}/wos_univ_paris8_{}.txt".format(str(year), str(year)), sep="\t",
                 quoting=csv.QUOTE_NONE, index_col=False, usecols=['DI'])
       wos_per_year.append(wos_df)

    full_wos = pandas.concat(wos_per_year)
    
    resultat = full_wos.rename (columns={'DI':column_name})
           
    return resultat

In [15]:
wos_df = read_wos(2016)

In [16]:
wos_df

,doi
0,10.1371/journal.pone.0168349
1,10.1016/j.chemgeo.2016.10.031
2,10.1016/j.jpowsour.2016.10.037
3,10.1016/j.jpowsour.2016.10.035
4,10.1371/journal.pone.0169282
...,...
3639,10.1109/TEM.2022.3171606
3640,10.1080/16258312.2022.2064721
3641,10.1007/s10479-021-04430-4
3642,10.1089/ham.2020.0184


Vérifier que le chiffre total obtenu ici correspond bien au chiffre trouvé dans le Web of Science pour toutes les années. Ne pas hésiter à refaire les extractions d'années précédentes si les chiffres ne correspondent pas : il peut y avoir de nouvelles publications indexées dans le Web of Science.

### Identifier les cellules qui n'ont pas de DOI

Trouver le nombre de lignes qui n'ont pas de DOI : utiliser la méthode **is not a number** (isna).

In [17]:
def wos_sans_doi():
    wos_df.isna().sum()
    return wos_df.isna().sum()

On met [0] pour demander le 1er élément de la liste

In [18]:
wos_sans_doi()[0]

3067

Trouver le nombre de lignes qui ont un DOI : utiliser la méthode **match** sur le string.

In [19]:
def wos_avec_doi():
    wos_df[column_name].str.match("10.").sum()
    return wos_df[column_name].str.match("10.").sum()

In [20]:
wos_avec_doi()

33576

Vérifier le nombre de lignes total :

In [21]:
len(wos_df)

36643

Vérifier qu'on a bien tout récupéré :

In [22]:
wos_sans_doi() + wos_avec_doi()

doi    36643
dtype: int64

Méthode de vérification de Python : **assert**. Si cela ne renvoie rien, c'est que les deux valeurs comparées sont identiques.

In [23]:
expected = len(wos_df[column_name])
actual = wos_sans_doi()[0] + wos_avec_doi()

assert(expected == actual)

 <span style="color:green"> **Les données du WoS sont à présent nettoyées et stockées dans la variable intitulée "wos_df".** </span>

## <span style="color:orange">Variante pour Scopus</span>

Ne pas exécuter cette partie si vous n'avez pas de fichiers Scopus et passer à la partie suivante.

In [ ]:
import pandas

In [ ]:
import csv

In [ ]:
with open("Data/raw/2016/scopus_lyon2_2016.csv") as f:
    scopus_string = f.read()

In [ ]:
scopus_string.split("\n") [1]

In [ ]:
scopus_df = pandas.read_csv("Data/raw/2016/scopus_lyon2_2016.csv", sep=',', encoding='latin-1')

In [ ]:
print(scopus_df.columns)

In [ ]:
import re

In [ ]:
scopus_doi = re.findall(r"(10\.\d{1,50}\S+)(\,)", scopus_string)

In [ ]:
scopus_doi[0]

Si, à la ligne "print(scopus_df.columns)", vous voyez 'DOI', alors exécutez le bloc de code ci-dessous, n'exécutez pas celui qui le suit immédiatement, et exécutez normalement à partir de la ligne scopus_df = read_scopus(2016).

Dans le cas contraire, n'exécutez pas le bloc de code ci-dessous mais exécutez plutôt le second, qui comprend la ligne "resultat_series = full_scopus_DOI.str.extract(r'(10\.\d{1,50}\S+)(\,)')[0]".

In [ ]:
def read_scopus(year):
    scopus_per_year = []
    for year in range (2016, 2022):
        scopus_df = pandas.read_csv("Data/raw/{}/scopus_lyon2_{}.csv".format(str(year), str(year))
                                    ,sep=",", encoding='utf-8')
        scopus_per_year.append(scopus_df)
    
    full_scopus = pandas.concat(scopus_per_year)
           
    resultat = pandas.DataFrame({column_name: full_scopus["DOI"]})                      
    return resultat

In [ ]:
def read_scopus(year):
    scopus_per_year = []
    for year in range (2016, 2022):
        scopus_df = pandas.read_csv("Data/raw/{}/scopus_lyon2_{}.csv".format(str(year), str(year)) ,sep=",", encoding='latin-1')
        scopus_per_year.append(scopus_df)
    
    full_scopus = pandas.concat(scopus_per_year)
    
    full_scopus_DOI = full_scopus["DOI"]
    resultat_series = full_scopus_DOI.str.extract(r'(10\.\d{1,50}\S+)(\,)')[0]
    resultat = pandas.DataFrame({column_name : resultat_series})                 
                          
    return resultat

In [ ]:
scopus_df = read_scopus(2016)

In [ ]:
scopus_df

## <span style="color:orange">Nettoyer les données issues de Pubmed</span>

In [4]:
with open("../Data/raw/2016/pubmed_univ_paris8_2016.csv", encoding='utf-8') as f:
    pubmed_string = f.read()

In [5]:
pubmed_string.split("\n") [1]

'10.18632/aging.101148,28039490,Impaired fasting blood glucose is associated to cognitive impairment and cerebral atrophy in middle-aged non-human primates.,Fathia_Djelti|Marc_Dhenain|Jérémy_Terrien|Jean-Luc_Picq|Isabelle_Hardy|Delphine_Champeval|Martine_Perret|Esther_Schenker|Jacques_Epelbaum|Fabienne_Aujard,2017-01-01,2016,,2017'

In [6]:
pubmed_df = pandas.read_csv("../Data/raw/2020/pubmed_univ_paris8_2020.csv", encoding='utf-8')

In [7]:
print(pubmed_df.columns)

Index(['DOI', 'PMID', 'title', 'auteurs', 'pub_date', 'Publication year',
       'published_date', 'pymed_publication_year'],
      dtype='object')


Il y a bien une colonne qui s'appelle "DOI", elle va être interrogée pour trouver directement les DOI. <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2024. Il faudra mettre 2026 à la place quand les données de 2025 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>


In [8]:
def read_pubmed(year):
    pubmed_per_year = []
    for year in range (2016, datetime.date.today().year):
        pubmed_df = pandas.read_csv("../Data/raw/{}/pubmed_univ_paris8_{}.csv".format(str(year), str(year))
                                    ,sep=",", encoding='utf-8')
        pubmed_per_year.append(pubmed_df)
    
    full_pubmed = pandas.concat(pubmed_per_year)
           
    resultat = pandas.DataFrame({column_name: full_pubmed["DOI"]})                      
    return resultat

In [9]:
pubmed_df = read_pubmed(2016)

In [10]:
pubmed_df

,doi
0,10.18632/aging.101148
1,10.3389/fpsyt.2016.00195
2,10.1111/sjop.12329
3,10.1007/s00221-016-4808-0
4,10.1080/01612840.2016.1224282
...,...
43,10.3758/s13421-024-01625-5
44,10.1111/jpm.13094
45,10.1037/dev0001790
46,10.1111/bjso.12790


### Vérifier qu'il ne manque pas de DOI

In [11]:
def pubmed_sans_doi():
    return len(pubmed_df.loc[pubmed_df.doi.isna()])

In [12]:
pubmed_sans_doi()

3

In [13]:
def pubmed_avec_doi():
    return len(pubmed_df.loc[~pubmed_df.doi.isna()])

In [14]:
pubmed_avec_doi()


439

In [15]:
expected = len(pubmed_df)
actual = pubmed_sans_doi() + pubmed_avec_doi()

print(expected == actual)

True


In [16]:
pubmed_df_avec_doi = pubmed_df.loc[~pubmed_df.doi.isna()]


 <span style="color:green"> **Les données de Pubmed sont à présent nettoyées et stockées dans la variable intitulée "pubmed_df".** </span>

## <span style="color:orange">Ajouter les données de HAL</span>

<span style="color:red"> Cette étape n'est plus nécessaire puisque le ou les codes collection HAL peuvent être transmis directement au MESR dans le fichier à envoyer lors d'une demande de BSO local.</span>

Ouvrir le fichier et concaténer les différentes années (mettre "2025" dans le code quand les données de 2024 seront injectées) :

In [19]:
def read_hal(year):
    hal_per_year = []
    for year in range (2016, datetime.date.today().year):
        hal_df = pandas.read_csv("../Data/raw/{}/hal_univ_paris8_{}.csv".format(str(year), str(year)), sep=",", encoding='latin-1')
        hal_per_year.append(hal_df)
    
    
    full_hal = pandas.concat(hal_per_year)
                     
    return full_hal

In [20]:
hal_df = read_hal(2016)

In [21]:
hal_df_doi = hal_df.loc[~hal_df.doiId_s.isna()]
hal_df_doi
doi_hal = hal_df_doi[["doiId_s"]].rename(columns={"doiId_s":"doi"})
doi_hal


,doi
6,10.4000/rfp.4975
8,10.1484/J.RHE.5.111948
12,10.1007/s11049-016-9353-9
17,10.4000/gss.3917
30,10.1038/srep29335
...,...
2176,10.56698/metropolitiques.2167
2177,10.3917/ds.492.0149
2178,10.3917/cdge.078.0009
2179,10.4000/14fwh


 <span style="color:green"> **Les données de HAL sont à présent nettoyées et stockées dans la variable intitulée "hal_df".** </span>

## <span style="color:orange">Rassembler les 2 listes de DOI (WoS, PubMed) et enlever les doublons</span>

Si un établissement utilise Scopus à la place du Web of Science, remplacer "wos_df" par "scopus_df".

### Concaténer les 2 listes de données

 <span style="color:red"> **Pour adapter ce code à votre établissement, vous pouvez changer le nom de la variable "all_univ_paris8" et y mettre à la place, par exemple, all_cote_azur. Mais attention dans ce cas à bien modifier le nom de la variable dans tout le reste du code.** </span>

In [22]:
all_univ_paris8 = pandas.concat([pubmed_df,doi_hal])

### Enlever les lignes sans DOI

In [23]:
import numpy

In [24]:
mask_doi = all_univ_paris8[column_name].notna()
mask_doi

0       True
1       True
2       True
3       True
4       True
        ... 
2176    True
2177    True
2178    True
2179    True
2181    True
Name: doi, Length: 10593, dtype: bool

Le résultat correspond à la concaténation de toutes les lignes des 5 fichiers, y compris celles qui ne comprennent pas de DOI.

In [25]:
doi_univ_paris8_doublons = all_univ_paris8[mask_doi]

In [26]:
doi_univ_paris8_doublons

,doi
0,10.18632/aging.101148
1,10.3389/fpsyt.2016.00195
2,10.1111/sjop.12329
3,10.1007/s00221-016-4808-0
4,10.1080/01612840.2016.1224282
...,...
2176,10.56698/metropolitiques.2167
2177,10.3917/ds.492.0149
2178,10.3917/cdge.078.0009
2179,10.4000/14fwh


Nous avons ici la liste de tous les DOI recensés dans nos 3 sources d'information. Mais attention, il peut y avoir des doublons : les DOI peuvent notamment être écrits en minuscules comme en majuscules.

### Enlever les doublons

Avant d'enlever les doublons, il faut normaliser les DOI et tous les passer en minuscules.

In [27]:
doi_univ_paris8_doublons_minuscule = doi_univ_paris8_doublons[column_name].str.lower()

In [28]:
doi_univ_paris8_doublons_df = pandas.DataFrame({column_name : doi_univ_paris8_doublons_minuscule}) 

Après avoir passé l'ensemble de la DataFrame en minuscules, on peut enlever les doublons.

In [29]:
doi_univ_paris8_final = doi_univ_paris8_doublons_df.drop_duplicates()

Réindexer la DataFrame pour que le nom des lignes soit propre :

In [30]:
doi_univ_paris8_final.reset_index(drop=True, inplace=True)

In [31]:
clean_doi = []
l_struct_id = []
l_nnt_etab = []
for n, x in enumerate(doi_univ_paris8_final.doi):
    if n == 0:
        l_struct_id.append("11141")
        l_nnt_etab.append("PA08")
    else:
        l_struct_id.append(None)
        l_nnt_etab.append(None)
    if re.search(".*doi.org/10.", x):
        remove_https = re.sub(".*doi.org/10.", "10.", x)
        clean_doi.append(remove_https)
        print(remove_https)
    else:
        clean_doi.append(x)
    

In [32]:
doi_univ_paris8_final = pandas.DataFrame(data={column_name : clean_doi})
fichier_envoye_au_MESR = pandas.DataFrame(data={column_name : clean_doi, "hal_struct_id":l_struct_id, "nnt_etab": l_nnt_etab})
fichier_envoye_au_MESR

,doi,hal_struct_id,nnt_etab
0,10.18632/aging.101148,11141,PA08
1,10.3389/fpsyt.2016.00195,None,None
2,10.1111/sjop.12329,None,None
3,10.1007/s00221-016-4808-0,None,None
4,10.1080/01612840.2016.1224282,None,None
...,...,...,...
9943,10.56698/metropolitiques.2167,None,None
9944,10.3917/ds.492.0149,None,None
9945,10.3917/cdge.078.0009,None,None
9946,10.4000/14fwh,None,None


**On obtient 32 178 DOI pour la période 2016-2024, sans compter HAL.**  <span style="color:red"> **Ce chiffre est à actualiser chaque année : dans le code, cela se fera automatiquement.** </span>

### Créer dans le dossier "outputs" un fichier CSV avec la liste des DOI

De même que précédemment, chaque établissement peut utiliser un nom différent : il faut simplement penser à remplacer "univ_paris8" par le nom de l'établissement à chaque fois qu'il apparaît dans le code et toujours l'écrire de la même façon.

In [33]:
doi_univ_paris8_final.to_csv("../Data/outputs/doi_univ_paris8.csv",index=False)
fichier_envoye_au_MESR.to_csv("../Data/fichier_envoye_au_MESR/BSO_univ_paris8_2025.csv",index=False, sep =";")

In [34]:
df = pandas.read_csv("../Data/outputs/doi_univ_paris8.csv", sep =",")
len(df)

9948

In [35]:
clean_doi = []
l_struct_id = []
l_nnt_etab = []
for n, x in enumerate(df.doi):
    if re.search("doi.org", x):
        remove_https = re.sub("doi.org/10.", "10.", x)
        clean_doi.append(remove_https)
        print(remove_https)
    else:
        clean_doi.append(x)

Pour obtenir cette liste dans un format Excel classique, exécuter la ligne de code suivante.

In [38]:
doi_univ_paris8_final.to_csv("Data\outputs\doi_univ_paris8.xls",index=False)

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_27928/2737923302.py:1: SyntaxWarning: invalid escape sequence '\o'
  doi_univ_paris8_final.to_csv("Data\outputs\doi_univ_paris8.xls",index=False)


Il faut ensuite se référer aux instructions présentes sur cette page : https://barometredelascienceouverte.esr.gouv.fr/declinaisons/comment-realiser-bso-local

In [40]:
import pandas as pd

In [42]:
df = pd.read_csv("../Data/fichier_from_MESR/bso-publications-latest_199318270_enriched.csv.gz", sep =";", dtype={'year':str})
df

,observation_date,id,doi,pmid,hal_id,year,title,journal_issns,journal_issn_l,journal_name,...,software_used,software_created,software_shared,data_used,data_created,data_shared,missing_doi_in_hal,has_doi_in_hal,doi_in_hal,bso_country
0,2024Q4,halhal-03933378,NaN,NaN,hal-03933378,2019,Typo-chronologie des tombes à inhumation en ré...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
1,2024Q4,halhal-03895949,NaN,NaN,hal-03895949,2017,Πήλινη σαρκοφάγος-λάρνακα από τη νεκρόπολη της...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
2,2024Q4,halhal-01789653,NaN,NaN,hal-01789653,2017,La naturaleza (sonora) de la ciudad. Buildings...,0719-630X,NaN,Panambi. Revista de Investigaciones Artísticas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
3,2024Q4,halhal-03807832,NaN,NaN,hal-03807832,2014,"L’ambivalente revendication d’exister, Mai 68 ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
4,2024Q4,doi10.3917/cliop.028.0029,10.3917/cliop.028.0029,NaN,hal-03819403,2022,Du cas Schreber et de ce qu’on ne doit pas ose...,2100-0670,2100-0670,Cliopsy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10.3917/cliop.028.0029,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,2024Q4,halhal-04095338,NaN,NaN,hal-04095338,2020,Décohabitation,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
45498,2024Q4,halhal-03299923,NaN,NaN,hal-03299923,2020,Scènes d’épouvante : Claude Simon dans les par...,"1774-9425,2558-782X",NaN,Cahiers Claude Simon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
45499,2024Q4,halhal-04366816,NaN,NaN,hal-04366816,2023,The Language of Bias: Unveiling Implicit Attit...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
45500,2024Q4,halhal-02120667,NaN,NaN,hal-02120667,2018,The archaeological project at Ra's al-Jinz RJ3...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr


In [44]:
df.loc[df.hal_id=="hal-02299291"]

,observation_date,id,doi,pmid,hal_id,year,title,journal_issns,journal_issn_l,journal_name,...,software_used,software_created,software_shared,data_used,data_created,data_shared,missing_doi_in_hal,has_doi_in_hal,doi_in_hal,bso_country
45461,2024Q4,halhal-02299291,NaN,NaN,hal-02299291,2018,Fictional Plays Broadcast on YouTube: Meet the...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fr
